## Titanic Assigment - Alicia Avis Bodas

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext(master = "local[4]")
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import IndexToString, StringIndexer


In [3]:
# Load training data
data = 'train.csv'
training = sqlContext.read.csv(data, header=True, inferSchema = True)
#training.toPandas().head()

# Load test data
datatest = 'test.csv'
testing = sqlContext.read.csv(datatest, header=True, inferSchema = True)
#testing.toPandas().head()

**Registramos la tabla con registerTempTable()**:

The reason to use the registerTempTable() method for a DataFrame, is so that in addition to being able to use the Spark-provided methods of a DataFrame, you can also issue SQL queries via the sqlContext.sql( sqlQuery ) method, that use that DataFrame as an SQL table.

In [8]:
training.registerTempTable("train")
testing.registerTempTable("test")

Con **StringIndexer** convertimos una string en un integer. En este caso, convertimos el valor de Sex en un código numérico bajo el nombre de IndexedSex.

En **handleInvalid** le decimos que tiene que hacer cuando encuentre un registro no válido. En este caso ponemos 'skip', que significa que cuando encuentre uno de estos registros lo "deje ahí y pase de ello".




In [13]:
sexindexer = StringIndexer(inputCol="Sex", outputCol="IndexedSex", handleInvalid= 'skip')
model = indexer.fit(training).transform(training) 
#sexindexer.toPandas()

Meto en una variable todas las características que voy a tener en cuenta para entrenar el modelo, todas ellas numéricas.

In [10]:
feat_num_col = ['Pclass', 'Age', 'IndexedSex','SibSp', 'Parch', 'Fare']

Transformo mediante **VectorAssembler** todas las características en un vector. Si no pongo handleInvalid= 'skip' no me deja hacer luego el transform.

In [15]:
assembler = VectorAssembler(inputCols= feat_num_col, outputCol="features", handleInvalid= 'skip')


Con el método **RandomForestClassifier**, establecemos en *labelCol* cual es la etiqueta que me va a permitir entrenar el modelo, en este caso es *survived*. 


In [25]:
rf = RandomForestClassifier(labelCol="Survived", 
                            featuresCol="features", 
                            predictionCol='prediction')

In [16]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier

Genero la **pipeline**, que es una transformación compuesta por otras transformaciones. También tiene fit y transform. Hago el fit con el trian y el transform con el test.

In [26]:
pipeline = Pipeline(stages=[sexindexer, assembler, rf])

In [27]:
model = pipeline.fit(training)

In [28]:
output = model.transform(testing)

Obtengo los datos de los pasajeros guardados para hacer el test, y me dice, según la predición, qué pasajero sobrevivirá.

In [37]:
#display(output.toPandas())

In [36]:
output.select("PassengerId", "prediction").show(truncate=False)

+-----------+----------+
|PassengerId|prediction|
+-----------+----------+
|892        |0.0       |
|893        |0.0       |
|894        |0.0       |
|895        |0.0       |
|896        |1.0       |
|897        |0.0       |
|898        |1.0       |
|899        |0.0       |
|900        |1.0       |
|901        |0.0       |
|903        |0.0       |
|904        |1.0       |
|905        |0.0       |
|906        |1.0       |
|907        |1.0       |
|908        |0.0       |
|909        |0.0       |
|910        |1.0       |
|911        |0.0       |
|912        |0.0       |
+-----------+----------+
only showing top 20 rows

